In [1]:
import pandas as pd
import joblib

from datetime import datetime as dtt
from forex_python.converter import CurrencyRates
from collections import Counter

## EDA

In [2]:
df = pd.read_excel('train_data/Hackatondata.xls')

In [3]:
# Select features for training
cols =['declaratie_nr', 'decl_datum_invoer', 'decl_datum_ingediend', 'huidig_declaratie_omschrijving', 
       'log_reden_wijziging', 'dde_omschrijving']
cols_dataset = ['declaratie_nr', 'decl_datum_invoer', 'decl_datum_ingediend', 'decl_valuta_code','decl_verw_bedr_val_vreem',
                'decl_verwacht_bedrag', 'dde_omschrijving', 'huidig_declaratie_status']
data = df[cols_dataset].drop_duplicates('declaratie_nr', keep='last')

data.decl_valuta_code.fillna(inplace=True, value='EUR')

mask = data.decl_verw_bedr_val_vreem.isna()
data.decl_verw_bedr_val_vreem[mask] = data.decl_verwacht_bedrag[mask].copy()

data.columns = ['ID', 'DatumInvoer', 'DatumIngediend', 'Valuta', 'BedragVreemd', 'BedragVerwacht', 'Categorie', 'Status']

data = data.dropna()

# types
data.DatumInvoer = pd.to_datetime(data.DatumInvoer)
data.DatumIngediend = pd.to_datetime(data.DatumIngediend)
data.Status = data.Status.astype(int)

In [4]:
a = list(data.Valuta.value_counts().head().index)

vmap = {'Parkeerkosten': 'Parking', 'Hotelkosten': 'Hotel', 'VOG / uittreksel GBA overig': 'VOG', 'Opleidingskosten': 'Certificatescosts', 'Reiskosten YP': 'Travelcosts'}

In [5]:
for i, r in data.iterrows():
    v = r['Categorie'] 
    v2 = r['Valuta']
    data.set_value(i, 'Categorie', vmap[v] if v in vmap else 'Other')
    data.set_value(i, 'Valuta',  v2 if v2 in a else 'Other')

/home/autarrsd/anaconda3/envs/side/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.
/home/autarrsd/anaconda3/envs/side/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """


In [52]:
data.Categorie.value_counts()
data.Valuta.value_counts()

EUR      6333
USD       422
Other     155
GBP        65
INR        64
SEK        56
Name: Valuta, dtype: int64

In [6]:
def useful(x):
    if x == 90:
        return True
    elif x > 90:
        return False
    else:
        return np.nan

In [7]:
data['Label'] = data.Status.apply(useful)

data = data.dropna(subset=['Label'])

data['DateDiff'] = (data.DatumIngediend - data.DatumInvoer).apply(lambda x: x.days)

data = data.drop(columns=['ID', 'Status', 'DatumInvoer', 'DatumIngediend'])

combined = pd.concat([data[['BedragVreemd', 'BedragVerwacht', 'Label', 'DateDiff']], pd.get_dummies(data.Valuta), 
           pd.get_dummies(data.Categorie)], axis=1)

### Classifier

In [59]:
len(X.columns)

15

In [58]:
X

,BedragVreemd,BedragVerwacht,DateDiff,EUR,GBP,INR,Other,SEK,USD,Certificatescosts,Hotel,Other,Parking,Travelcosts,VOG
14,350.00,291.06,408,0,0,0,0,0,1,1,0,0,0,0,0
32,531.84,443.20,335,0,0,0,0,0,1,0,1,0,0,0,0
42,20744.72,263.39,336,0,0,1,0,0,0,0,1,0,0,0,0
50,1058.00,13.43,336,0,0,1,0,0,0,0,0,1,0,0,0
56,1208.00,15.34,336,0,0,1,0,0,0,0,0,1,0,0,0
67,28.00,28.00,287,1,0,0,0,0,0,0,0,1,0,0,0
83,150.00,132.81,256,0,0,0,0,0,1,1,0,0,0,0,0
95,140.00,14.96,230,0,0,0,1,0,0,0,0,1,0,0,0
114,1159.00,1159.00,97,1,0,0,0,0,0,0,0,1,0,0,0
128,14.00,14.00,82,1,0,0,0,0,0,0,0,1,0,0,0


In [8]:
from imblearn.over_sampling import RandomOverSampler
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from xgboost import XGBRegressor 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

y = combined.Label.astype(int)
X = combined.drop(columns=['Label'])

ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X, y)
X_train, X_validation, y_train, y_validation = train_test_split(X_resampled, y_resampled, test_size=0.2,
                                                           random_state=42, stratify=y_resampled)

xgb = XGBClassifier(max_depth=25, n_estimators=500, n_jobs=8)
xgb.fit(X_train, y_train)

y_pred = xgb.predict(X_validation)
accuracy_score(y_validation, y_pred)

/home/autarrsd/anaconda3/envs/side/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


0.902734375

In [11]:
Counter(y_pred)

Counter({1: 1125, 0: 1435})

### Inference

In [9]:
res = pd.DataFrame()
res['Pred'] = y_pred
res['Prob'] = y_pred_proba[:,1]

NameError: name 'y_pred_proba' is not defined

In [111]:
len(res)

2560

In [82]:
len(res[(res.Prob < 0.2 ) | (res.Prob > 0.8)])

545

In [113]:
def def_action(x, thresh=0.8):
    if x > thresh:
        return 'Accept'
    elif x < 1-thresh:
        return 'Decline'
    else:
        return 'Expert'

In [115]:
res['Action'] = res.Prob.apply(def_action)

In [116]:
Counter(res.Action)

Counter({'Accept': 1067, 'Decline': 1260, 'Expert': 233})

### Save model

In [118]:
import joblib

In [119]:
# Saving model
joblib.dump(xgb, 'xgb_inference.joblib')

['xgb_inference.joblib']

In [120]:
# Loading model
xgb1 = joblib.load('xgb_inference.joblib')

In [124]:
X_validation

array([[128.65, 128.65,   0.  , ...,   0.  ,   0.  ,   0.  ],
       [255.2 , 255.2 ,   0.  , ...,   0.  ,   0.  ,   0.  ],
       [ 72.6 ,  72.6 ,   0.  , ...,   0.  ,   0.  ,   0.  ],
       ...,
       [ 59.  ,  59.  ,   0.  , ...,   0.  ,   0.  ,   0.  ],
       [ 46.13,  46.13,   0.  , ...,   0.  ,   0.  ,   0.  ],
       [ 18.15,  18.15,   0.  , ...,   0.  ,   0.  ,   0.  ]])

In [121]:
# Making sure it still works
y_pred = xgb1.predict(X_validation)
accuracy_score(y_validation, y_pred)

0.923828125

## Currency Rates

In [12]:
dtt.now()

datetime.datetime(2019, 5, 12, 4, 54, 5, 827404)

In [16]:
date_obj = dtt.now()
c = CurrencyRates()
date_obj

datetime.datetime(2019, 5, 12, 4, 54, 43, 127259)

In [18]:
c.get_ratem('USD','EUR', date_obj)

0.8904719501

In [2]:
% ls

 Volume in drive C is OS
 Volume Serial Number is 6CCF-598B

 Directory of C:\Users\work-02\Documents\git\hackdelft2019

11-May-19  04:37 PM    <DIR>          .
11-May-19  04:37 PM    <DIR>          ..
11-May-19  04:31 PM    <DIR>          .ipynb_checkpoints
11-May-19  04:33 PM        20,646,912 Hackatondata.xls
11-May-19  04:37 PM             1,300 Untitled.ipynb
               2 File(s)     20,648,212 bytes
               3 Dir(s)  14,602,547,200 bytes free
